<a href="https://colab.research.google.com/github/DavidMedeiros/lab5ri/blob/master/Lab5_RecInfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratório 5 - Expansão de Consultas
## David de Medeiros Souza

---


In [1]:
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import collections
from collections import Counter,OrderedDict
import csv
from tabulate import tabulate

import time
import heapq as hp

import nltk
from nltk.stem import RSLPStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Funções Auxiliares

In [0]:
"""
  Verifica se há numeros na string 
"""
def hasNumbers(inputString):
  return bool(re.search(r'\d', inputString))

""" 
  Retorna a quantidade de documentos que term aparece
"""
def num_of_docs_of_term(term, ranking):
  if(term in ranking.keys()):
    return len(ranking[term])
  else:
    return 0
  
"""
  Retorna a quantidade de documentos que o term_a E o term_b aparecem
"""
def num_docs_intercessions(term_a, term_b, ranking):
  if(term_a not in ranking.keys()):
    return 0
  if(term_b not in ranking.keys()):
    return 0
  
  docs1 = ranking[term_a]
  docs2 = ranking[term_b]
  counter = 0
  
  for doc in docs1:
    if doc in docs2:
      counter = counter + 1
      
  return counter

""" 
  Retorna o valor de Mutual information (MIM) para o term_a e term_n
""" 
def mutual_information(term_a, term_b, ranking):
  n_a = num_of_docs_of_term(term_a, ranking)
  n_b = num_of_docs_of_term(term_b, ranking)
  n_ab = num_docs_intercessions(term_a, term_b, ranking) 
   
  if (n_a * n_b) == 0:
    return 0
  return ((n_ab) / (n_a * n_b))

""" 
  Retorna o valor de Expected Mutual Information (EMIM) para o term_a e term_n
""" 
def expected_mutual_information(term_a, term_b, ranking, n):
  n_a = num_of_docs_of_term(term_a, ranking)
  n_b = num_of_docs_of_term(term_b, ranking)
  n_ab = num_docs_intercessions(term_a, term_b, ranking)
  
  if((n_a * n_b) == 0): 
    return 0
  
  if (n * ((n_ab) / (n_a * n_b))) == 0:
    return 0
 
  return n_ab * np.log(n * ((n_ab) / (n_a * n_b)))

""" 
  Retorna o valor de Chi Square para o term_a e term_n
""" 
def chi_square(term_a, term_b, ranking, n):
  n_a = num_of_docs_of_term(term_a, ranking)
  n_b = num_of_docs_of_term(term_b, ranking)
  n_ab = num_docs_intercessions(term_a, term_b, ranking)
  
  if (n_a * n_b) == 0:
    return 0
  
  return ((n_ab - (1/n) * n_a * n_b) ** 2) / (n_a * n_b)

""" 
  Retorna o valor de Dice’s coefficient (Dice) para o term_a e term_n
""" 
def dices_coefficient(term_a, term_b, ranking):
  n_a = num_of_docs_of_term(term_a, ranking)
  n_b = num_of_docs_of_term(term_b, ranking)
  n_ab = num_docs_intercessions(term_a, term_b, ranking)
    
  if (n_a + n_b) == 0:
    return 0
  
  return ((n_ab) / (n_a + n_b))

"""
  Algoritmo utilizado no laboratório anterior - Termo por vez
"""
def term_at_time(query, index, k):
  query_terms = query.split()    
  inverted_lists = []
  acumulador = {}

  for term in query_terms:
    if term in index.keys():
      inverted_lists.append(index[term])

  for lst in inverted_lists:
    for token,ocurrence in lst.items():
      d = token
      frequency = ocurrence
      if d in acumulador.keys():
        acumulador[d] = acumulador[d] + frequency
      else:
        acumulador[d] = frequency
  
  R = list(map(lambda elem: (elem[1],elem[0]), acumulador.items()))
  
  hp._heapify_max(R)
  
  top_k = []
  
  for i in range(1,k+1):
    if R != []:
      top = hp._heappop_max(R)
      top_k.append(top)
  
  return [(d, sd) for sd,d in top_k]

"""
  Retorna o top k documentos do index
"""
def top_k_docs(query, k):
  top_docs = [doc for doc,score in term_at_time(query, index, k)]
  top_docs_ranking = {}
  for termo in index:
    for document,score in index[termo].items():
      if document in top_docs:
        if termo in top_docs_ranking:
          top_docs_ranking[termo].append((document,score))
        else:
          top_docs_ranking[termo] = [(document, score)]
          
  return top_docs_ranking

"""
  Constói uma lista com os valores das métricas: MIM, EMIM, CHI-SQUARE e DICE
  Adaptada para ser utilizada também na questão 2
"""
def build_metric_list(query, metric, ranking, k):
  metric_list = []
  
  for termo in index.keys():
    if termo != query:
      if metric == 'mim':
        metric_value = mutual_information(query, termo, ranking)
      elif metric == 'emim':
        metric_value = expected_mutual_information(query, termo, ranking, k)
      elif metric == 'chi-square':
        metric_value = chi_square(query, termo, ranking, k)
      elif metric == 'dice':
        metric_value = dices_coefficient(query, termo, ranking)
        
      metric_list.append((termo, metric_value))      
 
  metric_list = sorted(metric_list, key = lambda x: x[1], reverse=True)
  
  return metric_list

"""
  Constrói uma tabela com as top-10 palavras mais associadas a cada delas de 
  acordo com as métricas MIM, EMIM, CHISQUARE e DICE.
"""
def get_table(termo, I, k):
  term_metrics = {}
  
  for query in queries:
    metrics = {}
    
    metrics['mim'] = build_metric_list(query, 'mim', I, k)
    metrics['emim'] = build_metric_list(query, 'emim', I, k)
    metrics['chi-square'] = build_metric_list(query, 'chi-square', I, k)
    metrics['dice'] = build_metric_list(query, 'dice', I, k)
    
    term_metrics[query] = metrics

  mim = [value for value,_ in term_metrics[termo]['mim'][0:10]]
  emim = [value for value,_ in term_metrics[termo]['emim'][0:10]]
  chi_square = [value for value,_ in term_metrics[termo]['chi-square'][0:10]]
  dice = [value for value,_ in term_metrics[termo]['dice'][0:10]]
  
  metrics_data = {'mim': mim, 'emim': emim, 'x2': chi_square, 'dice': dice}
  
  return pandas.DataFrame(metrics_data)

# Construção do índice 



In [0]:
dataset_url = 'https://raw.githubusercontent.com/DavidMedeiros/ri_lab_01/master/output/results.csv'
csv = pandas.read_csv(dataset_url)
documents = csv['text']

tokens = []
tokens_filtered= []

toker = RegexpTokenizer('''\w+[-']*\w*''')
stopwords = stopwords.words("portuguese")

for document in documents:
  tokens = tokens + toker.tokenize(document)


def build_index(documents):
  I = {}
  n = 0
  for document in documents:
    n += 1
    T = [token for token in toker.tokenize(document.lower())
         if token not in stopwords and len(token) > 2 and not hasNumbers(token)]

    for token in T:
      if token not in I:
        I[token] = {}
      
      ocurrence = T.count(token)
      if n not in I[token]:
        I[token][n] = ocurrence
      
  return I

index = build_index(documents)

data = {'token': list(index.keys()), 'ocurrences': list(index.values())}

df = pandas.DataFrame(data)

df.to_csv('index.csv')

# Questão 1

Considerando as consultas de um termo somente, utilizadas no laboratório anterior: **"ministro", "justiça", "bolsonaro", "lula" e "tribunal"**, essa questão tem como objetivo a construção de uma tabela para cada consulta, informando as top-10 palavras mais associadas a cada delas de acordo com as métricas MIM, EMIM, CHISQUARE e DICE.

Após visualizar as tabelas, acredito que **a métrica que obteve os melhores resultados foi o DICE**, pois as top-10 palavras são mais promissoras e fazem mais sentido para cada uma das cinco consultas. 


In [4]:
queries = ["ministro", "justiça", "bolsonaro", "lula", "tribunal"]

k = csv['text'].count()

for query in queries:
  print("TABELA PARA A CONSULTA: " + query) 
  print(tabulate(get_table(query, index, k) , headers='keys', tablefmt='fancy_grid'))
  print("\n")


TABELA PARA A CONSULTA: ministro
╒════╤════════════╤═════════╤════════════╤═════════╕
│    │ mim        │ emim    │ x2         │ dice    │
╞════╪════════════╪═════════╪════════════╪═════════╡
│  0 │ perfil     │ desta   │ guedes     │ desta   │
├────┼────────────┼─────────┼────────────┼─────────┤
│  1 │ verificada │ federal │ desta      │ federal │
├────┼────────────┼─────────┼────────────┼─────────┤
│  2 │ poucas     │ guedes  │ ricardo    │ sobre   │
├────┼────────────┼─────────┼────────────┼─────────┤
│  3 │ provar     │ israel  │ publicação │ após    │
├────┼────────────┼─────────┼────────────┼─────────┤
│  4 │ postando   │ após    │ quanto     │ vai     │
├────┼────────────┼─────────┼────────────┼─────────┤
│  5 │ calendário │ sobre   │ israel     │ israel  │
├────┼────────────┼─────────┼────────────┼─────────┤
│  6 │ datando    │ ricardo │ rodriguez  │ paulo   │
├────┼────────────┼─────────┼────────────┼─────────┤
│  7 │ provando   │ oficial │ negócios   │ guedes  │
├────┼───────

# Questão 2
De acordo com a métrica que deu os melhores resultados na questão anterior **DICE**, agora executaremos cada consulta (usando a abordagem termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. 

**Após visualizar as tabelas, podemos perceber que a qualidade dos resultados tende a melhorar quando o parâmetro K aumenta, visto que, a ligação entre as palavras será maior. Assim, o top10 possui palavras mais significativas do que o top3.**

Expansão para os 
***TOP 3 Elementos***

In [5]:
top_k = 3

for query in queries:
  ranking_docs = top_k_docs(query, k)
  
  table_dice = pandas.DataFrame()
  table_dice['DICE before'] = get_table(query, index, k)['dice']
  table_dice['DICE after'] = get_table(query, ranking_docs, top_k)['dice']
  
  print('Top-10 termos associados aos Top-3 Documentos para a query: ' + query) 
  print(tabulate(table_dice , headers='keys', tablefmt='fancy_grid'))
  print('\n')


Top-10 termos associados aos Top-3 Documentos para a query: ministro
╒════╤═══════════════╤══════════════╕
│    │ DICE before   │ DICE after   │
╞════╪═══════════════╪══════════════╡
│  0 │ desta         │ disse        │
├────┼───────────────┼──────────────┤
│  1 │ federal       │ anos         │
├────┼───────────────┼──────────────┤
│  2 │ sobre         │ ainda        │
├────┼───────────────┼──────────────┤
│  3 │ após          │ jair         │
├────┼───────────────┼──────────────┤
│  4 │ vai           │ twitter      │
├────┼───────────────┼──────────────┤
│  5 │ israel        │ todos        │
├────┼───────────────┼──────────────┤
│  6 │ paulo         │ estados      │
├────┼───────────────┼──────────────┤
│  7 │ guedes        │ vai          │
├────┼───────────────┼──────────────┤
│  8 │ brasil        │ resistência  │
├────┼───────────────┼──────────────┤
│  9 │ disse         │ econômica    │
╘════╧═══════════════╧══════════════╛


Top-10 termos associados aos Top-3 Documentos para a qu

Expansão para os 
***TOP 5 Elementos***

In [6]:
top_k = 5

for query in queries:
  ranking_docs = top_k_docs(query, k)
  
  table_dice = pandas.DataFrame()
  table_dice['DICE before'] = get_table(query, index, k)['dice']
  table_dice['DICE after'] = get_table(query, ranking_docs, top_k)['dice']
  
  print('Top-10 termos associados aos Top-5 Documentos para a query: ' + query) 
  print(tabulate(table_dice , headers='keys', tablefmt='fancy_grid'))
  print('\n')

Top-10 termos associados aos Top-5 Documentos para a query: ministro
╒════╤═══════════════╤══════════════╕
│    │ DICE before   │ DICE after   │
╞════╪═══════════════╪══════════════╡
│  0 │ desta         │ disse        │
├────┼───────────────┼──────────────┤
│  1 │ federal       │ anos         │
├────┼───────────────┼──────────────┤
│  2 │ sobre         │ ainda        │
├────┼───────────────┼──────────────┤
│  3 │ após          │ jair         │
├────┼───────────────┼──────────────┤
│  4 │ vai           │ twitter      │
├────┼───────────────┼──────────────┤
│  5 │ israel        │ todos        │
├────┼───────────────┼──────────────┤
│  6 │ paulo         │ estados      │
├────┼───────────────┼──────────────┤
│  7 │ guedes        │ vai          │
├────┼───────────────┼──────────────┤
│  8 │ brasil        │ resistência  │
├────┼───────────────┼──────────────┤
│  9 │ disse         │ econômica    │
╘════╧═══════════════╧══════════════╛


Top-10 termos associados aos Top-5 Documentos para a qu

Expansão para os 
***TOP 10 Elementos***

In [7]:
top_k = 10

for query in queries:
  ranking_docs = top_k_docs(query, k)
  
  table_dice = pandas.DataFrame()
  table_dice['DICE before'] = get_table(query, index, k)['dice']
  table_dice['DICE after'] = get_table(query, ranking_docs, top_k)['dice']
  
  print('Top-10 termos associados aos Top-10 Documentos para a query: ' + query) 
  print(tabulate(table_dice , headers='keys', tablefmt='fancy_grid'))
  print('\n')

Top-10 termos associados aos Top-10 Documentos para a query: ministro
╒════╤═══════════════╤══════════════╕
│    │ DICE before   │ DICE after   │
╞════╪═══════════════╪══════════════╡
│  0 │ desta         │ disse        │
├────┼───────────────┼──────────────┤
│  1 │ federal       │ anos         │
├────┼───────────────┼──────────────┤
│  2 │ sobre         │ ainda        │
├────┼───────────────┼──────────────┤
│  3 │ após          │ jair         │
├────┼───────────────┼──────────────┤
│  4 │ vai           │ twitter      │
├────┼───────────────┼──────────────┤
│  5 │ israel        │ todos        │
├────┼───────────────┼──────────────┤
│  6 │ paulo         │ estados      │
├────┼───────────────┼──────────────┤
│  7 │ guedes        │ vai          │
├────┼───────────────┼──────────────┤
│  8 │ brasil        │ resistência  │
├────┼───────────────┼──────────────┤
│  9 │ disse         │ econômica    │
╘════╧═══════════════╧══════════════╛


Top-10 termos associados aos Top-10 Documentos para a 